# JUICE RPWI HF SID2 (RAW): L1a QL -- 2024/2/17

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import copy
import csv

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_0-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib  as juice_cdf
import juice_sid2_lib as juice_sid2
# import juice_math_lib as juice_math

# Setting and Read CDF file: set by User

In [ ]:
# *** Plot dump ***
dump_mode = 1                           # 0: no-dump  1:plot dump

# *** CAL set ***
band_mode = 0                           # [Power]      0: sum      1: /Hz
unit_mode = 0                           # [Power]      0: raw      1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI
cal_mode = 2                            # [Power]      0: background     1: CAL     2; all

# *** Unit set ***
f_mode = 1                              # [Frequency]  0: linear   1:log  in frequency
time_mode = 0                           # [Time]       0: Epoch    1: data number
# spec_mode = 1                         # 0: low-resoltion   1: high-resolution
# ave_mode = 2                          # 0: simple sum      1: FFT sum     2: median sum   3: min sum
# clean_mode = 0                        # 0: normal  1: cleaning in time   2: cleaning in time & frequency

# *** Frenquency range set ***
f_mode_min = 3000
f_mode_max = 10000

# *** Filter make ***
# filter_mode = 0                         # 0: read     1: make

# *** Power range set ***
p_raw_max = 7.0                         #              background: 7.5   CAL: 10
p_raw_min = 0.0                         #              background: 2.5   CAL: 5
cf, p_max0, p_min0 = juice_cdf.cal_factors(unit_mode, p_raw_max, p_raw_min)
print("conversion factor:", cf, "    MAX-min:", p_max0, p_min0)

# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

In [ ]:
# *** Ground - Ver.2 ***
"""
# 202310 -- SAMPLE
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample/cdf/'        # CDF data folder
data_name_list = ['SID02_20231007-0349.cdf',
                  #'SID02_20231117-1607.cdf',
                 ]
data_name_list = ['SID02-high_20231019-1749.cdf',
                  'SID02-high_20231020-1218.cdf',
                  'SID02-high_20231020-1222.cdf',
                 ]

# 202312 -- TEST
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202312_FS/cdf/'    # 202312_FS test folder
data_name_list = ['SID2_20000101T004732-20000101T004929_0.cdf',
                  'SID2_20000101T004934-20000101T005147_1.cdf',
                  'SID2_20000101T005151-20000101T005357_2.cdf',
                  'SID2_20000101T005401-20000101T005538_3.cdf',
                  'SID2_20000101T005543-20000101T005624_4.cdf',
                 ]

# 202312 -- Checkout
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202312_C/cdf/'    # 202312_FS test folder
data_name_list = [#'SID02_RPWI_HF_FFT_00000.cdf',
                  #'SID02_RPWI_HF_FFT_00001.cdf',
                  #'SID02_RPWI_HF_FFT_00002.cdf',
                  #'SID02_RPWI_HF_FFT_00003.cdf',
                  #'SID02_RPWI_HF_FFT_00004.cdf',
                  'SID02_RPWI_S8_272.cdf',
                 ]

# 202401 -- ErrorCheck
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202401_FS/cdf/'    # 202312_FS test folder
data_name_list = [#'JUICE_L1a_RPWI-HF-SID2_FSW1_1F_0_jumps.bin.cdf',
                  #'JUICE_L1a_RPWI-HF-SID2_FSW1_1F_21_no_jump.bin.cdf',
                  'JUICE_L1a_RPWI-HF-SID2_high.cdf',
                 ]
"""

In [ ]:
# *** Flight data: Ver.2 ***
# 20240126
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/2024/01/25/'        # CDF data folder
data_name_list = ['JUICE_L1a_RPWI-HF-SID2_20240125T112327_V01.cdf',      # 128
                  #'JUICE_L1a_RPWI-HF-SID2_20240125T152238_V01.cdf',      # 512
                 ]
"""
"""

In [ ]:
# *** Flight - Ver.1
"""
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/2023/'        # CDF data folder

# *** 20230419 ***
data_name_list = ['04/19/SID2_20230419T135849-20230419T141229.cdf',
                  '04/19/SID2_20230419T141231-20230419T141402.cdf',
                 ]

# *** 20230530 ***
data_name_list = ['05/30/SID2_20230530T100326-20230530T100925.cdf',
                  '05/30/SID2_20230530T100927-20230530T100937.cdf',
                 ]
data_name_list = ['05/30/JUICE_L1a_RPWI-HF-SID2_20230530T100326-20230530T100925_V01___RPR1_52000010_2023.150.10.40.53.663.cdf',
                  '05/30/JUICE_L1a_RPWI-HF-SID2_20230530T100927-20230530T100937_V01___RPR1_52000011_2023.150.10.41.53.508.cdf',
                 ]
                 # *** 20230601 ***
data_name_list = ['06/01/SID2_20230601T120759-20230601T120857.cdf',
                  '06/01/SID2_20230601T121435-20230601T121533.cdf',
                  '06/01/SID2_20230601T122138-20230601T122236.cdf',
                  '06/01/SID2_20230601T122707-20230601T122805.cdf',
                 ]

# *** all: 202305-06 ***
data_name_list = ['05/30/SID2_20230530T100326-20230530T100925.cdf',
                  '05/30/SID2_20230530T100927-20230530T100937.cdf',
                  '06/01/SID2_20230601T120759-20230601T120857.cdf',
                  '06/01/SID2_20230601T121435-20230601T121533.cdf',
                  '06/01/SID2_20230601T122138-20230601T122236.cdf',
                  '06/01/SID2_20230601T122707-20230601T122805.cdf',
                 ]

# *** 20230712-13 ***
data_name_list = ['07/12/SID2_20230712T090434-20230712T093848.cdf',
                  '07/12/SID2_20230712T093942-20230712T101355.cdf',
                  '07/12/SID2_20230712T101449-20230712T104147.cdf',
                 ]
data_name_list = ['07/12/SID2_20230712T104149-20230712T232406.cdf']
#
data_name_list = ['07/12/SID2_20230712T232408-20230712T235156.cdf']
# type-III
data_name_list = ['07/12/SID2_20230712T235158-20230713T001854.cdf',
                  '07/13/SID2_20230713T001856-20230713T004644.cdf',
                 ]
data_name_list = ['07/13/SID2_20230713T023720-20230713T030416.cdf',
                 ]
data_name_list = ['07/12/SID2_20230712T232408-20230712T235156.cdf',
                  '07/12/SID2_20230712T235158-20230713T001854.cdf',
                  '07/13/SID2_20230713T001856-20230713T004644.cdf',
                  '07/13/SID2_20230713T004648-20230713T011342.cdf',
                  '07/13/SID2_20230713T011436-20230713T014134.cdf',
                  '07/13/SID2_20230713T014136-20230713T020924.cdf',
                  '07/13/SID2_20230713T020928-20230713T023718.cdf',
                  '07/13/SID2_20230713T023720-20230713T030416.cdf',
                  '07/13/SID2_20230713T030510-20230713T033208.cdf',
                  '07/13/SID2_20230713T033210-20230713T040000.cdf',
                  '07/13/SID2_20230713T040002-20230713T042751.cdf',
                  '07/13/SID2_20230713T042753-20230713T045449.cdf',
                  '07/13/SID2_20230713T045543-20230713T050917.cdf',
                 ]
# all
data_name_list = ['07/12/SID2_20230712T090434-20230712T093848.cdf',
                  '07/12/SID2_20230712T093942-20230712T101355.cdf',
                  '07/12/SID2_20230712T101449-20230712T104147.cdf',
                  '07/12/SID2_20230712T104149-20230712T232406.cdf',
                  '07/12/SID2_20230712T232408-20230712T235156.cdf',
                  '07/12/SID2_20230712T235158-20230713T001854.cdf',
                  '07/13/SID2_20230713T001856-20230713T004644.cdf',
                  '07/13/SID2_20230713T004648-20230713T011342.cdf',
                  '07/13/SID2_20230713T011436-20230713T014134.cdf',
                  '07/13/SID2_20230713T014136-20230713T020924.cdf',
                  '07/13/SID2_20230713T020928-20230713T023718.cdf',
                  '07/13/SID2_20230713T023720-20230713T030416.cdf',
                  '07/13/SID2_20230713T030510-20230713T033208.cdf',
                  '07/13/SID2_20230713T033210-20230713T040000.cdf',
                  '07/13/SID2_20230713T040002-20230713T042751.cdf',
                  '07/13/SID2_20230713T042753-20230713T045449.cdf',
                  '07/13/SID2_20230713T045543-20230713T050917.cdf',
                 ]

# ALL
data_name_list = ['05/30/SID2_20230530T100326-20230530T100925.cdf',
                  '05/30/SID2_20230530T100927-20230530T100937.cdf',
                  '06/01/SID2_20230601T120759-20230601T120857.cdf',
                  '06/01/SID2_20230601T121435-20230601T121533.cdf',
                  '06/01/SID2_20230601T122138-20230601T122236.cdf',
                  '06/01/SID2_20230601T122707-20230601T122805.cdf',
                  '07/12/SID2_20230712T090434-20230712T093848.cdf',
                  '07/12/SID2_20230712T093942-20230712T101355.cdf',
                  '07/12/SID2_20230712T101449-20230712T104147.cdf',
                  '07/12/SID2_20230712T104149-20230712T232406.cdf',
                  '07/12/SID2_20230712T232408-20230712T235156.cdf',
                  '07/12/SID2_20230712T235158-20230713T001854.cdf',
                  '07/13/SID2_20230713T001856-20230713T004644.cdf',
                  '07/13/SID2_20230713T004648-20230713T011342.cdf',
                  '07/13/SID2_20230713T011436-20230713T014134.cdf',
                  '07/13/SID2_20230713T014136-20230713T020924.cdf',
                  '07/13/SID2_20230713T020928-20230713T023718.cdf',
                  '07/13/SID2_20230713T023720-20230713T030416.cdf',
                  '07/13/SID2_20230713T030510-20230713T033208.cdf',
                  '07/13/SID2_20230713T033210-20230713T040000.cdf',
                  '07/13/SID2_20230713T040002-20230713T042751.cdf',
                  '07/13/SID2_20230713T042753-20230713T045449.cdf',
                  '07/13/SID2_20230713T045543-20230713T050917.cdf',
                 ]
"""

In [ ]:
# *** Ground - Ver.1
"""
# *** 202105 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202105/'        # CDF data folder
data_name_list = ['SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32770.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32774.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32775.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32776.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32777.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32778.cdf',
                 ]
data_name_list = ['SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32772.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32773.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32774.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32775.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32776.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32777.cdf',
                 ]
# all
data_name_list = ['SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32770.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32774.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32775.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32776.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32777.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32778.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32772.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32773.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32774.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32775.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32776.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32777.cdf',
                 ]
                 
# *** 202106 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202106/'        # CDF data folder
data_name_list = ['SID2_SCTBTV_Phase11_xid32831.cdf',
                  'SID2_SCTBTV_Phase11_xid32832.cdf',
                  'SID2_SCTBTV_Phase11_xid32833.cdf',
                  'SID2_SCTBTV_Phase11_xid32834.cdf',
                 ]
data_name_list = ['SID2_SCTBTV_Phase13_xid32844.cdf',
                  'SID2_SCTBTV_Phase13_xid32845.cdf',
                  'SID2_SCTBTV_Phase13_xid32846.cdf',
                 ]
# all
data_name_list = ['SID2_SCTBTV_Phase11_xid32831.cdf',
                  'SID2_SCTBTV_Phase11_xid32832.cdf',
                  'SID2_SCTBTV_Phase11_xid32833.cdf',
                  'SID2_SCTBTV_Phase11_xid32834.cdf',
                  'SID2_SCTBTV_Phase13_xid32844.cdf',
                  'SID2_SCTBTV_Phase13_xid32845.cdf',
                  'SID2_SCTBTV_Phase13_xid32846.cdf',
                 ]

# *** 202111 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202111/'        # CDF data folder
data_name_list = ['SID2_SCPFM_PTR_RPWI_delta.RPWI_SCM_TEST_xid32770.cdf',
                  'SID2_SCPFM_PTR_RPWI_delta.RPWI_SCM_TEST_xid32771.cdf',
                  'SID2_SCPFM_PTR_RPWI_delta.RPWI_SCM_TEST_xid32772.cdf',
                 ]
data_name_list = ['SID2_SCPFM_RPWI_30c_xid32776.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32777.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32778.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32779.cdf',
                 ]
# all
data_name_list = ['SID2_SCPFM_PTR_RPWI_delta.RPWI_SCM_TEST_xid32770.cdf',
                  'SID2_SCPFM_PTR_RPWI_delta.RPWI_SCM_TEST_xid32771.cdf',
                  'SID2_SCPFM_PTR_RPWI_delta.RPWI_SCM_TEST_xid32772.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32776.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32777.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32778.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32779.cdf',
                 ]

# *** 202207 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202207/'        # CDF data folder
data_name_list = ['SID2_SCPFM_RPWI_30c_xid32776.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32777.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32778.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32779.cdf',
                 ]

# *** 202208 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202208/'        # CDF data folder
data_name_list = ['SID2_20220824_HF-FFT-rerun_xid32791.cdf',
                  'SID2_20220824_HF-FFT-rerun_xid32792.cdf',
                  'SID2_20220824_HF-FFT-rerun_xid32793.cdf',
                  ]
"""

In [ ]:
# *** Group read
"""
import glob
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample/cdf/'        # CDF data folder
data_name = 'SID02*.cdf'
cdf_file = data_dir + data_name
data_name_list = glob.glob(cdf_file)
num_list = len(data_name_list)
data_name_list.sort()
for i in range(num_list):
    data_name_list[i] = os.path.split(data_name_list[i])[1]
"""

# get CDF data

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)

for i in range(num_list):
    data_name = data_name_list[i]
    cdf_file = data_dir + data_name
    print(i, cdf_file)

    cdf = pycdf.CDF(cdf_file)
    data1 = juice_sid2.hf_sid2_read(cdf)

    if i==0:
        data = data1
        print(data.Eu_i.shape)
    else:
        data = juice_sid2.hf_sid2_add(data, data1)
        print(data.Eu_i.shape)

print("path name:", data_name);  data_name = os.path.split(data_name)[1];  print("data name:", data_name)

In [ ]:
data = juice_sid2.hf_sid2_proc(data)
date1 = data.epoch[0]
date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]
date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
print(str_date)

n_time0 = data.Eu_i.shape[0]
n_freq0 = data.Eu_i.shape[1]
n_samp0 = data.Eu_i.shape[2]
f_min0 = data.frequency[0][0][0]
f_max0 = (max(np.ravel(data.frequency)))

In [ ]:
# Mode 
N_ch0 = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]
print("Ch:", N_ch0, "  (U:", data.U_selected[0], "  V:", data.V_selected[0], "  W:", data.W_selected[0], ")")
print("Num-samples:", n_time0, "   Num-Frequency:", n_freq0, "   Length:", n_samp0)
print("Frequency, width, step (kHz):", f_min0,  "-", f_max0, data.freq_width[0][0][0], data.freq_step[0][0][0])
print("Time-length:", data.time[0][0][n_samp0-1], "sec in 1-sweep")
# print(data.epoch)

In [ ]:
# Mask read
n_mask = 19392
mask_list = np.zeros(n_mask * 3)
mask_list = mask_list.reshape(n_mask, 3)
mask_dir = './lib/'
mask_name = 'hf_mask_natural.csv'
mask_file = mask_dir + mask_name

# mask_natural read
with open(mask_file, 'r') as f:
    reader = csv.reader(f)
    i = 0
    for row in reader:
        mask_list[i, 0] = row[1];  mask_list[i, 1] = row[2];  mask_list[i, 2] = row[3]
        i = i+1
print("mask read: ", mask_list.shape, mask_file)

# Raw data

In [ ]:
fig = plt.figure(figsize=(12, 14))
ax1 = fig.add_subplot(6, 1, 1);  ax2 = fig.add_subplot(6, 1, 2);  ax3 = fig.add_subplot(6, 1, 3)
ax4 = fig.add_subplot(6, 1, 4);  ax5 = fig.add_subplot(6, 1, 5);  ax6 = fig.add_subplot(6, 1, 6)

ax1.plot(np.ravel(data.Eu_i[:][:]), '-r', linewidth=.5, label='Eu_i'); ax1.plot(np.ravel(data.Eu_q[:][:]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(data.Ev_i[:][:]), '-r', linewidth=.5, label='Ev_i'); ax2.plot(np.ravel(data.Ev_q[:][:]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(data.Ew_i[:][:]), '-r', linewidth=.5, label='Ew_i'); ax3.plot(np.ravel(data.Ew_q[:][:]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency),  '-b', linewidth=.5, label='Frequency');
ax4.plot(np.ravel(data.freq_step*10),  '-g', linewidth=0.8, label='step*10')
ax4.plot(np.ravel(data.freq_width*10), ':b', linewidth=1.0, label='width*10')
ax4.plot(np.ravel(data.sweep_start)*data.frequency[0][-1][0], '-r', linewidth=.5, label='Sweep Start');
ax5.plot(np.ravel(data.T_HF_FPGA),  ':r', label='T (HK-FPGA)')
ax5.plot(np.ravel(data.T_RWI_CH1),  ':b', label='T (RWI1)')
ax5.plot(np.ravel(data.T_RWI_CH2),  ':g', label='T (RWI2)')
ax5.plot(np.ravel(data.cal_signal*10), '-k', label='CAL')
ax6.plot(np.ravel(data.epoch[:]), '.')

ax1.set_ylabel('Eu');  ax2.set_ylabel('Ev');  ax3.set_ylabel('Ew');  ax4.set_ylabel('Frequency [kHz]');  ax5.set_ylabel('T [degC]');  
ax6.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF RAW (SID-2)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[0, len(np.ravel(data.Eu_i[:][:]))];  print(xlim)
ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
"""
# range: Y-axis
#ylim=[-10**(p_max0), 10**(p_max0)]
#ax1.set_ylim(ylim);  ax2.set_ylim(ylim);  ax3.set_ylim(ylim)
ylim=[f_min0, f_max0]; ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

# print(data.T_HF_FPGA, data.T_RWI_CH1, data.T_RWI_CH2)


## RAW - First 2 Sweeps

In [ ]:
n_sweep0 = 0;    n_sweep1 = n_sweep0 + 10
print("[specific sweep]  SWEEP:", n_sweep0, n_sweep1, "in total of", n_time0)
f_min = f_min0;  f_max = f_max0

fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(5, 1, 1);  ax2 = fig.add_subplot(5, 1, 2);  ax3 = fig.add_subplot(5, 1, 3);  ax4 = fig.add_subplot(5, 1, 4)
ax5 = fig.add_subplot(5, 1, 5)

ax1.plot(np.ravel(data.Eu_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Eu_i'); 
ax1.plot(np.ravel(data.Eu_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(data.Ev_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Ev_i'); 
ax2.plot(np.ravel(data.Ev_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(data.Ew_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Ew_i'); 
ax3.plot(np.ravel(data.Ew_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency[n_sweep0:n_sweep1]),  '-b', linewidth=.5, label='Frequency');
ax4.plot(np.ravel(data.sweep_start[n_sweep0:n_sweep1])*data.frequency[n_sweep0][-1][0], '-r', label='Sweep Start')
ax5.plot(np.ravel(data.epoch[n_sweep0:n_sweep1+1]), '.')
ax1.set_ylabel('Eu');  ax2.set_ylabel('Ev');  ax3.set_ylabel('Ew');  ax4.set_ylabel('Frequency [kHz]');  
date1 = data.epoch[n_sweep0];  date1 = date1.strftime('%Y/%m/%d %R:%S');  ax4.set_xlabel(date1)
#
title_label = '[JUICE/RPWI HF RAW (SID-2)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[len(np.ravel(data.Eu_i[n_sweep0:n_sweep1]))//2 - 32, len(np.ravel(data.Eu_i[n_sweep0:n_sweep1]))//2 + 32];  print(xlim)
ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
"""
# range: Y-axis
ylim=[-10**(p_max0-5), 10**(p_max0-5)]; ax1.set_ylim(ylim);  ax2.set_ylim(ylim);  ax3.set_ylim(ylim)
"""
"""
ylim=[f_min, f_max]; ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-sweep.png'
    fig.savefig(png_fname)

## Raw - First 3 steps

In [ ]:
n_sweep = 0  # n_time0//2  
n_step0 = 0;  n_step1 = n_step0 + 3
print("[specific sweep]  SWEEP:", n_sweep, "in total of", n_time0,  "[Steps]", n_step0, "-", n_step1)

fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(np.ravel(data.Eu_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Eu_i')
ax1.plot(np.ravel(data.Eu_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(data.Ev_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Ev_i')
ax2.plot(np.ravel(data.Ev_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(data.Ew_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Ew_i')
ax3.plot(np.ravel(data.Ew_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency[n_sweep][n_step0:n_step1]),  '-b', linewidth=.5, label='Frequency')
ax4.plot(np.ravel(data.sweep_start[n_sweep][n_step0:n_step1])*data.frequency[n_sweep][n_step1][0], '-r', label='Sweep Start')
ax1.set_ylabel('Eu');  ax2.set_ylabel('Ev');  ax3.set_ylabel('Ew');  ax4.set_ylabel('Frequency [kHz]');  
date1 = data.epoch[n_sweep];  date1 = date1.strftime('%Y/%m/%d %R:%S');  ax4.set_xlabel(date1)
#
title_label = '[JUICE/RPWI HF RAW (SID-2)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[0, len(np.ravel(data.Eu_i[:][:]))];  print(xlim)
ax1.set_xlim(xlim);  ax3.set_xlim(xlim);  ax5.set_xlim(xlim);  ax7.set_xlim(xlim)
"""
# range: Y-axis
"""
ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)]
ax1.set_ylim(ylim);  ax3.set_ylim(ylim);  ax5.set_ylim(ylim)
"""
#ylim=[f_min, f_max]; ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-step.png'
    fig.savefig(png_fname)

# Get Spectrum

In [ ]:
# Spec mode: high resolution  
spec = juice_sid2.hf_sid2_getspec(data, cal_mode)
n_time0 = spec.EE.shape[0]
EE_2d   = spec.EE.transpose();         EuEu_2d = spec.EuEu.transpose();         EvEv_2d = spec.EvEv.transpose();         EwEw_2d = spec.EwEw.transpose()
# Median formation
EE_med     = np.median(spec.EE, axis=0); EuEu_med = np.median(spec.EuEu, axis=0); EvEv_med = np.median(spec.EvEv, axis=0); EwEw_med = np.median(spec.EwEw, axis=0)

# index
power_str = juice_cdf.power_label(band_mode, unit_mode)
freq_1d = spec.freq[n_time0//2]  # freq_width_1d  = spec.freq_width[0]
n_freq0 = freq_1d.shape[0]
num_1d = np.arange(n_time0)

Epoch_1d = spec.epoch.tolist()

"""
freq_width_2d  = data.freq_width.transpose()
if band_mode == 1:
    EuEu_2d = EuEu_2d / freq_width_2d / 1000
    EvEv_2d = EvEv_2d / freq_width_2d / 1000
    EwEw_2d = EwEw_2d / freq_width_2d / 1000
"""

print(power_str, "EuEu:", spec.EuEu.shape, freq_1d[0], "-", freq_1d[-1], "kHz  (df =", freq_1d[1] - freq_1d[0], "kHz)" )

## CAL

In [ ]:
"""
# cal_unit = juice_cdf.cal_unit(unit_mode, freq_1d)
num_freq = freq_1d.shape[0]
for i in range(freq_1d.shape[0]):
    print(freq_1d[i])
"""    

In [ ]:
# spec = juice_sid2.hf_sid2_speccal_unit(spec, unit_mode)

# Spectrum

In [ ]:
# Sweep_num
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
n_sweep1 = n_time0//4;  n_sweep2 = 3*n_time0//4;  n_sweep3 = n_time0-1
# n_sweep1 = 50; n_sweep2 = 1600;  n_sweep3 = 2000
# Y-range
p_min = p_min0;   p_max = p_max0
# X-range
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(freq_1d, spec.EE[n_sweep1], '-r', linewidth=.9, label='EE 1st')
ax2.plot(freq_1d, spec.EuEu[n_sweep1], '-r', linewidth=.9, label='EuEu 1st')
ax3.plot(freq_1d, spec.EvEv[n_sweep1], '-r', linewidth=.9, label='EvEv 1st')
ax4.plot(freq_1d, spec.EwEw[n_sweep1], '-r', linewidth=.9, label='EwEw 1st')
ax1.plot(freq_1d, spec.EE[n_sweep2], '-g', linewidth=.4, label='EE mid')
ax2.plot(freq_1d, spec.EuEu[n_sweep2], '-g', linewidth=.4, label='EuEu mid')
ax3.plot(freq_1d, spec.EvEv[n_sweep2], '-g', linewidth=.4, label='EvEv mid')
ax4.plot(freq_1d, spec.EwEw[n_sweep2], '-g', linewidth=.4, label='EwEw mid')
ax1.plot(freq_1d, spec.EE[n_sweep3], '-b', linewidth=.2, label='EE last')
ax2.plot(freq_1d, spec.EuEu[n_sweep3], '-b', linewidth=.2, label='EuEu last')
ax3.plot(freq_1d, spec.EvEv[n_sweep3], '-b', linewidth=.2, label='EvEv last')
ax4.plot(freq_1d, spec.EwEw[n_sweep3], '-b', linewidth=.2, label='EwEw last')
ax1.set_yscale('log'); ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log')

# Label
ax1.set_ylabel('EE');  ax2.set_ylabel('EuEu'); ax3.set_ylabel('EvEv'); ax4.set_ylabel('EwEw'); ax4.set_xlabel('Frequency [kHz]')
#
date1 = Epoch_1d[n_sweep1];  date1 = date1.strftime('1st: %Y/%m/%d %R:%S   ')
date2 = Epoch_1d[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = Epoch_1d[n_sweep3];  date3 = date3.strftime('Fin: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min, f_max]
# xlim=[8150, 8250]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim)

# range: Y-axis
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log.png'
    fig.savefig(png_fname)

# Peak
print(f"[First - {n_sweep1}]")
peak_E = np.ravel(spec.EE[n_sweep1]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
print(f"[Mid - {n_sweep2}]")
peak_E = np.ravel(spec.EE[n_sweep2]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
print(f"[Last - {n_sweep3}]")
peak_E = np.ravel(spec.EE[n_sweep3]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")

In [ ]:
# Sweep_num
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
n_sweep1 = n_time0//4;  n_sweep2 = n_time0//2;  n_sweep3 = 3*n_time0//4
# n_sweep1 = n_time0//4;  n_sweep2 = 3*n_time0//4;  n_sweep3 = n_time0-1
# Y-range
p_min = p_min0;   p_max = p_max0-2
# X-range
f_min = 0;  f_max = 5000
# f_min = 190;  f_max = 210

fig = plt.figure(figsize=(12, 20))
ax1 = fig.add_subplot(9, 1, 1);  ax2 = fig.add_subplot(9, 1, 2);  ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4)
ax5 = fig.add_subplot(9, 1, 5);  ax6 = fig.add_subplot(9, 1, 6);  ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8)
ax9 = fig.add_subplot(9, 1, 9)

ax1.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax1.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE med')
ax1.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax1.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax2.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax2.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE med')
ax2.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax2.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax3.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax3.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE med')
ax3.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax3.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax4.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax4.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE med')
ax4.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax4.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax5.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax5.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE med')
ax5.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax5.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax6.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax6.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE med')
ax6.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax6.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax7.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax7.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE med')
ax7.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax7.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax8.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax8.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE med')
ax8.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax8.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax9.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax9.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE med')
ax9.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax9.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax1.set_yscale('log');  ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log');  ax5.set_yscale('log')
ax6.set_yscale('log');  ax7.set_yscale('log');  ax8.set_yscale('log');  ax9.set_yscale('log')

# Mask - natural
"""
ax1.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask'); ax2.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask')
ax3.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask'); ax4.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask')
ax5.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask'); ax6.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask')
ax7.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask'); ax8.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask')
ax9.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask')
"""

# Label
ax1.set_ylabel(power_str); ax2.set_ylabel(power_str); ax3.set_ylabel(power_str); ax4.set_ylabel(power_str); ax5.set_ylabel(power_str)
ax6.set_ylabel(power_str); ax7.set_ylabel(power_str); ax8.set_ylabel(power_str); ax9.set_ylabel(power_str)
ax9.set_xlabel('Frequency [kHz]')
#
date1 = Epoch_1d[n_sweep1];  date1 = date1.strftime(f'1st {n_sweep1}: %Y/%m/%d %R:%S   ')
date2 = Epoch_1d[n_sweep2];  date2 = date2.strftime(f'Mid {n_sweep2}: %Y/%m/%d %R:%S   ')
date3 = Epoch_1d[n_sweep3];  date3 = date3.strftime(f'Fin {n_sweep3}: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)

ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);  ax6.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8);  ax8.legend(loc='upper right', fontsize=8);  ax9.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min,       f_max];        ax1.set_xlim(xlim); xlim=[f_min+5000,  f_max+5000];   ax2.set_xlim(xlim);  
xlim=[f_min+10000, f_max+10000];  ax3.set_xlim(xlim); xlim=[f_min+15000, f_max+15000];  ax4.set_xlim(xlim)
xlim=[f_min+20000, f_max+20000];  ax5.set_xlim(xlim); xlim=[f_min+25000, f_max+25000];  ax6.set_xlim(xlim)
xlim=[f_min+30000, f_max+30000];  ax7.set_xlim(xlim); xlim=[f_min+35000, f_max+35000];  ax8.set_xlim(xlim)
xlim=[f_min+40000, f_max+40000];  ax9.set_xlim(xlim)

# range
ylim=[10**p_min, 10**p_max]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim); ax5.set_ylim(ylim); ax6.set_ylim(ylim);  
ax7.set_ylim(ylim); ax8.set_ylim(ylim);  ax9.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_wo-msk.png'
    fig.savefig(png_fname)

# Frequency - Time diagram

In [ ]:
# Y-range
f_min = f_min0;      f_max = f_max0
# f_min = 41900;      f_max = 42100
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])
# Z-range
p_min  = p_min0;     p_max  = p_max0
#p_min  = p_min0+1.5;     p_max  = p_max0-2.5

fig2d = plt.figure(figsize=[12,16])
ax1 = fig2d.add_subplot(5, 1, 1);  ax2 = fig2d.add_subplot(5, 1, 2); ax3 = fig2d.add_subplot(5, 1, 3); ax4 = fig2d.add_subplot(5, 1, 4)
if time_mode == 1:
    ax5 = fig2d.add_subplot(5, 1, 5)

# Y-axis
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log');        ax4.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]'); ax4.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title('EE');  ax2.set_title('EuEu');  ax3.set_title('EvEv'); ax4.set_title('EwEw')
if time_mode == 1:
    ax5.set_xlabel(str_date)
else:
    ax4.set_xlabel(str_date)

# Plot
print(num_1d.shape, freq_1d.shape, EE_2d.shape)
if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EE_2d,   norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.pcolormesh(num_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p5 = ax5.plot(np.ravel(spec.epoch[:]), '.')
    pp4 = fig2d.colorbar(p4, ax=ax5, orientation="vertical"); pp4.set_label(power_str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_2d,   norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.pcolormesh(Epoch_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(power_str)
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical"); pp4.set_label(power_str)

# range: X-axis
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log.png'
    fig2d.savefig(png_fname)

In [ ]:
# Y-range
f_min = f_min0;      f_max = f_max0
f_min = 100;         f_max = 1000
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])
# Z-range
p_min  = p_min0;     p_max  = p_max0
p_min  = p_min0+1.5;     p_max  = p_max0-2.5

fig2d = plt.figure(figsize=[12,8])
ax1 = fig2d.add_subplot(1, 1, 1);  
if time_mode == 1:
    ax1 = fig2d.add_subplot(1, 1, 1);  ax5 = fig2d.add_subplot(2, 1, 2)

# Y-axis
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title('EE')
if time_mode == 1:
    ax5.set_xlabel(str_date)
else:
    ax1.set_xlabel(str_date)

# Plot
print(num_1d.shape, freq_1d.shape, EE_2d.shape)
if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EE_2d,   norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p5 = ax5.plot(np.ravel(spec.epoch[:]), '.')
    pp1 = fig2d.colorbar(p1, ax=ax5, orientation="vertical"); pp1.set_label(power_str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_2d,   norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(power_str)

# range: X-axis
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT-EE.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-EE-log.png'
    fig2d.savefig(png_fname)

# Mask filter - natural

In [ ]:
"""
# Mask read
n_mask = 19392
mask_list = np.zeros(n_mask * 3)
mask_list = mask_list.reshape(n_mask, 3)
mask_dir = './lib/'
mask_name = 'hf_mask_natural.csv'
mask_file = mask_dir + mask_name

# mask_natural read
with open(mask_file, 'r') as f:
    reader = csv.reader(f)
    i = 0
    for row in reader:
        mask_list[i, 0] = row[1];  mask_list[i, 1] = row[2];  mask_list[i, 2] = row[3]
        i = i+1
print("mask_natural read: ", mask_list.shape, mask_file)
"""

In [ ]:
# mask adopt
spec_msk = copy.deepcopy(spec)

for i in range(freq_1d.shape[0]):
    num_mask = np.int16((freq_1d[i] - 80) // 2.3125)
    if (num_mask >= 0 and num_mask < n_mask):
        if mask_list[num_mask,2] > 0:
            spec_msk.EE[:,i-1] = 0
            spec_msk.EE[:,i] = 0
            if i<freq_1d.shape[0]-1:
                spec_msk.EE[:,i+1] = 0

# delete
mask_index0    = np.where(spec_msk.EE[0] == 0)
spec_msk.EE   = np.delete(spec_msk.EE, mask_index0, 1)
spec_msk.EuEu = np.delete(spec_msk.EuEu, mask_index0, 1)
spec_msk.EvEv = np.delete(spec_msk.EvEv, mask_index0, 1)
spec_msk.EwEw = np.delete(spec_msk.EwEw, mask_index0, 1)
freq_1d_msk   = np.delete(freq_1d, mask_index0, 0)

# Median
EE_msk   = np.median(spec_msk.EE, axis=0)
EuEu_msk = np.median(spec_msk.EuEu, axis=0)
EvEv_msk = np.median(spec_msk.EvEv, axis=0)
EwEw_msk = np.median(spec_msk.EwEw, axis=0)

# 2D
EE_msk_2d = spec_msk.EE.transpose() 
EuEu_msk_2d = spec_msk.EuEu.transpose() 
EvEv_msk_2d = spec_msk.EvEv.transpose() 
EwEw_msk_2d = spec_msk.EwEw.transpose() 

## Spectrum - mask cleaned

In [ ]:
# Sweep_num
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
n_sweep1 = n_time0//4;  n_sweep2 = 3*n_time0//4;  n_sweep3 = n_time0-1
# n_sweep1 = 50; n_sweep2 = 1600;  n_sweep3 = 2000
# Y-range
p_min = p_min0;   p_max = p_max0+1
# X-range
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(freq_1d_msk, spec_msk.EE[n_sweep1], '-r', linewidth=.9, label='EE 1st')
ax2.plot(freq_1d_msk, spec_msk.EuEu[n_sweep1], '-r', linewidth=.9, label='EuEu 1st')
ax3.plot(freq_1d_msk, spec_msk.EvEv[n_sweep1], '-r', linewidth=.9, label='EvEv 1st')
ax4.plot(freq_1d_msk, spec_msk.EwEw[n_sweep1], '-r', linewidth=.9, label='EwEw 1st')
ax1.plot(freq_1d_msk, spec_msk.EE[n_sweep2], '-g', linewidth=.4, label='EE mid')
ax2.plot(freq_1d_msk, spec_msk.EuEu[n_sweep2], '-g', linewidth=.4, label='EuEu mid')
ax3.plot(freq_1d_msk, spec_msk.EvEv[n_sweep2], '-g', linewidth=.4, label='EvEv mid')
ax4.plot(freq_1d_msk, spec_msk.EwEw[n_sweep2], '-g', linewidth=.4, label='EwEw mid')
ax1.plot(freq_1d_msk, spec_msk.EE[n_sweep3], '-b', linewidth=.2, label='EE last')
ax2.plot(freq_1d_msk, spec_msk.EuEu[n_sweep3], '-b', linewidth=.2, label='EuEu last')
ax3.plot(freq_1d_msk, spec_msk.EvEv[n_sweep3], '-b', linewidth=.2, label='EvEv last')
ax4.plot(freq_1d_msk, spec_msk.EwEw[n_sweep3], '-b', linewidth=.2, label='EwEw last')
ax1.set_yscale('log'); ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log')

# Label
ax1.set_ylabel('EE '+power_str);   ax2.set_ylabel('EuEu '+power_str); ax3.set_ylabel('EvEv '+power_str); 
ax4.set_ylabel('EwEw '+power_str); ax4.set_xlabel('Frequency [kHz]')
#
date1 = Epoch_1d[n_sweep1];  date1 = date1.strftime('1st: %Y/%m/%d %R:%S   ')
date2 = Epoch_1d[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = Epoch_1d[n_sweep3];  date3 = date3.strftime('Fin: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min, f_max]
# xlim=[8150, 8250]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim)

# range: Y-axis
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-msk.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-msk-log.png'
    fig.savefig(png_fname)

# Peak
print(f"[First - {n_sweep1}]")
peak_E = np.ravel(spec_msk.EE[n_sweep1]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d_msk[peak_f]), "kHz")
peak_E = np.ravel(spec_msk.EuEu[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d_msk[peak_f]), "kHz")
peak_E = np.ravel(spec_msk.EvEv[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d_msk[peak_f]), "kHz")
peak_E = np.ravel(spec_msk.EwEw[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d_msk[peak_f]), "kHz")
print(f"[Mid - {n_sweep2}]")
peak_E = np.ravel(spec_msk.EE[n_sweep2]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d_msk[peak_f]), "kHz")
peak_E = np.ravel(spec_msk.EuEu[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d_msk[peak_f]), "kHz")
peak_E = np.ravel(spec_msk.EvEv[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d_msk[peak_f]), "kHz")
peak_E = np.ravel(spec_msk.EwEw[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d_msk[peak_f]), "kHz")
print(f"[Last - {n_sweep3}]")
peak_E = np.ravel(spec_msk.EE[n_sweep3]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d_msk[peak_f]), "kHz")
peak_E = np.ravel(spec_msk.EuEu[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d_msk[peak_f]), "kHz")
peak_E = np.ravel(spec_msk.EvEv[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d_msk[peak_f]), "kHz")
peak_E = np.ravel(spec_msk.EwEw[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d_msk[peak_f]), "kHz")

In [ ]:
# Sweep_num
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
n_sweep1 = n_time0//4;  n_sweep2 = 3*n_time0//4;  n_sweep3 = n_time0-1
# Y-range
p_min = p_min0-2;   p_max = p_max0
# X-range
f_min = 0;  f_max = 5000

fig = plt.figure(figsize=(12, 20))
ax1 = fig.add_subplot(9, 1, 1);  ax2 = fig.add_subplot(9, 1, 2);  ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4)
ax5 = fig.add_subplot(9, 1, 5);  ax6 = fig.add_subplot(9, 1, 6);  ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8)
ax9 = fig.add_subplot(9, 1, 9)

ax1.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave'); ax1.plot(freq_1d_msk, EE_msk, '-b', linewidth=1, label='EE mask')
ax2.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave'); ax2.plot(freq_1d_msk, EE_msk, '-b', linewidth=1, label='EE mask')
ax3.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave'); ax3.plot(freq_1d_msk, EE_msk, '-b', linewidth=1, label='EE mask')
ax4.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave'); ax4.plot(freq_1d_msk, EE_msk, '-b', linewidth=1, label='EE mask')
ax5.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave'); ax5.plot(freq_1d_msk, EE_msk, '-b', linewidth=1, label='EE mask')
ax6.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave'); ax6.plot(freq_1d_msk, EE_msk, '-b', linewidth=1, label='EE mask')
ax7.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave'); ax7.plot(freq_1d_msk, EE_msk, '-b', linewidth=1, label='EE mask')
ax8.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave'); ax8.plot(freq_1d_msk, EE_msk, '-b', linewidth=1, label='EE mask')
ax9.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave'); ax9.plot(freq_1d_msk, EE_msk, '-b', linewidth=1, label='EE mask')
ax1.set_yscale('log');  ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log');  ax5.set_yscale('log')
ax6.set_yscale('log');  ax7.set_yscale('log');  ax8.set_yscale('log');  ax9.set_yscale('log')

# Mask - natural
ax1.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask'); ax2.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask')
ax3.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask'); ax4.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask')
ax5.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask'); ax6.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask')
ax7.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask'); ax8.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask')
ax9.plot(mask_list[:,0], mask_list[:,2]*15, '.r', label='mask')

# Label
ax1.set_ylabel(power_str); ax2.set_ylabel(power_str); ax3.set_ylabel(power_str); ax4.set_ylabel(power_str); ax5.set_ylabel(power_str)
ax6.set_ylabel(power_str); ax7.set_ylabel(power_str); ax8.set_ylabel(power_str); ax9.set_ylabel(power_str)
# Label
ax1.set_ylabel('EE '+power_str);   ax2.set_ylabel('EuEu '+power_str); ax3.set_ylabel('EvEv '+power_str); 
ax4.set_ylabel('EwEw '+power_str); ax4.set_xlabel('Frequency [kHz]')
ax9.set_xlabel('Frequency [kHz]')
#
date1 = Epoch_1d[n_sweep1];  date1 = date1.strftime(f'1st {n_sweep1}: %Y/%m/%d %R:%S   ')
date2 = Epoch_1d[n_sweep2];  date2 = date2.strftime(f'Mid {n_sweep2}: %Y/%m/%d %R:%S   ')
date3 = Epoch_1d[n_sweep3];  date3 = date3.strftime(f'Fin {n_sweep3}: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)

ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);  ax6.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8);  ax8.legend(loc='upper right', fontsize=8);  ax9.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min,       f_max];        ax1.set_xlim(xlim); 
xlim=[f_min+5000,  f_max+5000];   ax2.set_xlim(xlim);  
xlim=[f_min+10000, f_max+10000];  ax3.set_xlim(xlim); 
xlim=[f_min+15000, f_max+15000];  ax4.set_xlim(xlim)
xlim=[f_min+20000, f_max+20000];  ax5.set_xlim(xlim)
xlim=[f_min+25000, f_max+25000];  ax6.set_xlim(xlim)
xlim=[f_min+30000, f_max+30000];  ax7.set_xlim(xlim)
xlim=[f_min+35000, f_max+35000];  ax8.set_xlim(xlim)
xlim=[f_min+40000, f_max+40000];  ax9.set_xlim(xlim)

# range: Y-axis
"""
ylim=[p_min*10, p_max*10]
ylim=[1, 50]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim); ax5.set_ylim(ylim); ax6.set_ylim(ylim);  
ax7.set_ylim(ylim); ax8.set_ylim(ylim);  ax9.set_ylim(ylim)
"""

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_w-msk.png'
    fig.savefig(png_fname)

## FT - mask cleaned

In [ ]:
# Y-range
f_min = f_min0;      f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])
# Z-range
p_min  = p_min0+1;     p_max  = p_max0-4

fig2d = plt.figure(figsize=[12,16])
ax1 = fig2d.add_subplot(5, 1, 1);  ax2 = fig2d.add_subplot(5, 1, 2); ax3 = fig2d.add_subplot(5, 1, 3); ax4 = fig2d.add_subplot(5, 1, 4)
if time_mode == 1:
    ax5 = fig2d.add_subplot(5, 1, 5)

# Y-axis
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log');        ax4.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]'); ax4.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title('EE');  ax2.set_title('EuEu');  ax3.set_title('EvEv'); ax4.set_title('EwEw')
if time_mode == 1:
    ax5.set_xlabel(str_date)
else:
    ax4.set_xlabel(str_date)

# Plot
print(num_1d.shape, freq_1d.shape, EE_2d.shape)
if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d_msk, EE_msk_2d,   norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d_msk, EuEu_msk_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d_msk, EvEv_msk_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.pcolormesh(num_1d, freq_1d_msk, EwEw_msk_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p5 = ax5.plot(np.ravel(Epoch_1d[:]), '.')
    pp4 = fig2d.colorbar(p4, ax=ax5, orientation="vertical"); pp4.set_label(power_str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d_msk, EE_msk_2d,   norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d_msk, EuEu_msk_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d_msk, EvEv_msk_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.pcolormesh(Epoch_1d, freq_1d_msk, EwEw_msk_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(power_str)
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical"); pp4.set_label(power_str)

# range: X-axis
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT-msk.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-msk-log.png'
    fig2d.savefig(png_fname)

# SID-3 emulation

In [ ]:
# SID-3: Frequency
asw_ver = 2
freq_1d_sid, f_step_sid, f_width_sid = juice_cdf._frequency_sid3(asw_ver)
n_freq1_sid = len(freq_1d_sid)
n_freq1 = len(freq_1d)
n_freq1_msk = len(freq_1d_msk)
tbl_freq_to_sid     = juice_cdf._frequency_sid2_to_data(freq_1d_sid, f_step_sid, freq_1d)
tbl_freq_to_sid_msk = juice_cdf._frequency_sid2_to_data(freq_1d_sid, f_step_sid, freq_1d_msk)

spec_sid_EE          = np.zeros((n_time0, n_freq1_sid))
spec_sid_EE_msk      = np.zeros((n_time0, n_freq1_sid))
spec_sid_EE_med      = np.zeros((n_time0, n_freq1_sid))
spec_sid_EE_msk_med  = np.zeros((n_time0, n_freq1_sid))
spec_sid_EE_mean     = np.zeros((n_time0, n_freq1_sid))
spec_sid_EE_msk_mean = np.zeros((n_time0, n_freq1_sid))
#print(spec.EE.shape, n_time0, n_freq1)
#print(spec_msk.EE.shape, n_time0, n_freq1_msk, spec.EE.shape, spec_msk.EE.shape, n_freq1, n_freq1_msk)

for i in range(n_time0):
    for j in range(n_freq1_sid):
        j0 = tbl_freq_to_sid[j][0]; j0_msk = tbl_freq_to_sid_msk[j][0]
        j1 = tbl_freq_to_sid[j][1]; j1_msk = tbl_freq_to_sid_msk[j][1]
        j2 = tbl_freq_to_sid[j][2]; j2_msk = tbl_freq_to_sid_msk[j][2]
        spec_sid_EE[i][j]          = np.sum(spec.EE[i][j0:j1+1])
        spec_sid_EE_msk[i][j]      = np.sum(spec_msk.EE[i][j0_msk:j1_msk+1])
        spec_sid_EE_med[i][j]      = np.median(spec.EE[i][j0:j1+1])
        spec_sid_EE_msk_med[i][j]  = np.median(spec_msk.EE[i][j0_msk:j1_msk+1])
        spec_sid_EE_mean[i][j]     = np.mean(spec.EE[i][j0:j1+1])
        spec_sid_EE_msk_mean[i][j] = np.mean(spec_msk.EE[i][j0_msk:j1_msk+1])
        if i==1 and j>100:
            print('{:3d}'.format(j), '{:.2e}'.format(freq_1d_sid[j]), '{:4d}'.format(j0), '{:4d}'.format(j1), '{:2d}'.format(j2), '{:.2e}'.format(spec_sid_EE[i][j]), '{:.2e}'.format(spec_sid_EE_med[i][j]), '{:.2e}'.format(spec_sid_EE_mean[i][j]))
            print('{:3d}'.format(j), '{:.2e}'.format(freq_1d_sid[j]), '{:4d}'.format(j0_msk), '{:4d}'.format(j1_msk), '{:2d}'.format(j2_msk), '{:.2e}'.format(spec_sid_EE_msk[i][j]), '{:.2e}'.format(spec_sid_EE_msk_med[i][j]), '{:.2e}'.format(spec_sid_EE_msk_mean[i][j]))

EE_sid_2d      = spec_sid_EE.transpose();       EE_sid_2d_msk      = spec_sid_EE_msk.transpose()
EE_sid_2d_med  = spec_sid_EE_med.transpose();   EE_sid_2d_msk_med  = spec_sid_EE_msk_med.transpose()
EE_sid_2d_mean = spec_sid_EE_mean.transpose();  EE_sid_2d_msk_mean = spec_sid_EE_msk_mean.transpose()
"""
mask_file = './test.csv'
with open(mask_file, 'w') as f:
    writer = csv.writer(f)
    for i in range(n_freq1_sid):
        f_min = freq_1d_sid[i] - f_step_sid[i]/2
        f_max = freq_1d_sid[i] + f_step_sid[i]/2
        writer.writerow([ i, f_min, f_max, tbl_freq_to_sid[i, 0], tbl_freq_to_sid[i, 1], tbl_freq_to_sid[i, 2], tbl_freq_to_sid_msk[i, 0], tbl_freq_to_sid_msk[i, 1], tbl_freq_to_sid_msk[i, 2] ])
print("sample write: ", mask_list.shape, mask_file)
"""

In [ ]:
# Sweep_num
n_sweep1 = n_time0//4
# Y-range
p_min = p_min0;  p_max = p_max0+1
# X-range
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(freq_1d,     spec.EE[n_sweep1],              ':r', linewidth=.9, label='EE(SID-2) 1st')
ax1.plot(freq_1d_sid, spec_sid_EE[n_sweep1],          '-r', linewidth=.9, label='EE(SID-3) 1st')
ax1.plot(freq_1d_sid, spec_sid_EE_mean[n_sweep1],     '-g', linewidth=.9, label='EE_mean(SID-3) 1st')
ax1.plot(freq_1d_sid, spec_sid_EE_med[n_sweep1],      '-b', linewidth=.9, label='EE_med(SID-3) 1st')
#
ax2.plot(freq_1d_msk, spec_msk.EE[n_sweep1],          ':r', linewidth=.9, label='EE_msk(SID-2) 1st')
ax2.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep1],      '-r', linewidth=.9, label='EE_msk(SID-3) 1st')
ax2.plot(freq_1d_sid, spec_sid_EE_msk_mean[n_sweep1], '-g', linewidth=.9, label='EE_msk_mean(SID-3) 1st')
ax2.plot(freq_1d_sid, spec_sid_EE_msk_med[n_sweep1],  '-b', linewidth=.9, label='EE_msk_med(SID-3) 1st')
#
ax3.plot(freq_1d,     spec.EE[n_sweep1],              ':r', linewidth=.9, label='EE(SID-2) 1st')
ax3.plot(freq_1d_msk, spec_msk.EE[n_sweep1],          '-b', linewidth=.9, label='EE_msk(SID-2) 1st')
#
ax4.plot(freq_1d_sid, spec_sid_EE[n_sweep1],          ':r', linewidth=.9, label='EE(SID-3) 1st')
ax4.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep1],      '-r', linewidth=.9, label='EE_msk(SID-3) 1st')
ax4.plot(freq_1d_sid, spec_sid_EE_mean[n_sweep1],     ':g', linewidth=.9, label='EE_mean(SID-3) 1st')
ax4.plot(freq_1d_sid, spec_sid_EE_msk_mean[n_sweep1], '-g', linewidth=.9, label='EE_msk_mean(SID-3) 1st')
ax4.plot(freq_1d_sid, spec_sid_EE_med[n_sweep1],      ':b', linewidth=.9, label='EE_med(SID-3) 1st')
ax4.plot(freq_1d_sid, spec_sid_EE_msk_med[n_sweep1],  '-b', linewidth=.9, label='EE_msk_med(SID-3) 1st')
ax1.set_yscale('log'); ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log')

# Label
ax1.set_ylabel('EE '+power_str);       ax2.set_ylabel('EE-msk '+power_str)
ax3.set_ylabel('EE:SID-2 '+power_str); ax4.set_ylabel('EE:SID-3 emu '+power_str); ax4.set_xlabel('Frequency [kHz]')
#
date1 = Epoch_1d[n_sweep1];  date1 = date1.strftime('1st: %Y/%m/%d %R:%S   ')
date2 = Epoch_1d[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = Epoch_1d[n_sweep3];  date3 = date3.strftime('Fin: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min, f_max]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim)

# range: Y-axis
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-sid3emu.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-sid3emu-log.png'
    fig.savefig(png_fname)

### Spectrum - SID-3 emulation

In [ ]:
# Sweep_num
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
n_sweep1 = n_time0//4;  n_sweep2 = 3*n_time0//4;  n_sweep3 = n_time0-1
# Y-range
p_min = p_min0;  p_max = p_max0+1
# X-range
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])
#   f_min = 3900;        f_max = 5100;        print(data.frequency[0][0][0], data.frequency[0][8][0])

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.9, label='EE(SID-2) 1st')
ax1.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.5, label='EE(SID-2) mid')
ax1.plot(freq_1d, spec.EE[n_sweep3], ':k', linewidth=.5, label='EE(SID-2) last')
ax1.plot(freq_1d_sid, spec_sid_EE[n_sweep1], '-r', linewidth=.9, label='EE(SID-3) 1st')
ax1.plot(freq_1d_sid, spec_sid_EE[n_sweep2], '-g', linewidth=.5, label='EE(SID-3) mid')
ax1.plot(freq_1d_sid, spec_sid_EE[n_sweep3], '-k', linewidth=.5, label='EE(SID-3) last')
#
ax2.plot(freq_1d_msk, spec_msk.EE[n_sweep1], ':r', linewidth=.9, label='EE_msk(SID-2) 1st')
ax2.plot(freq_1d_msk, spec_msk.EE[n_sweep2], ':g', linewidth=.5, label='EE_msk(SID-2) mid')
ax2.plot(freq_1d_msk, spec_msk.EE[n_sweep3], ':k', linewidth=.5, label='EE_msk(SID-2) last')
ax2.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep1], '-r', linewidth=.9, label='EE_msk(SID-3) 1st')
ax2.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep2], '-g', linewidth=.5, label='EE_msk(SID-3) mid')
ax2.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep3], '-k', linewidth=.5, label='EE_msk(SID-3) last')
#
ax3.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.9, label='EE(SID-2) 1st')
ax3.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.5, label='EE(SID-2) mid')
ax3.plot(freq_1d, spec.EE[n_sweep3], ':k', linewidth=.3, label='EE(SID-2) last')
ax3.plot(freq_1d_msk, spec_msk.EE[n_sweep1], '-r', linewidth=.9, label='EE_msk(SID-2) 1st')
ax3.plot(freq_1d_msk, spec_msk.EE[n_sweep2], '-g', linewidth=.5, label='EE_msk(SID-2) mid')
ax3.plot(freq_1d_msk, spec_msk.EE[n_sweep3], '-k', linewidth=.5, label='EE_msk(SID-2) last')
#
ax4.plot(freq_1d_sid, spec_sid_EE[n_sweep1], ':r', linewidth=.9, label='EE(SID-3) 1st')
ax4.plot(freq_1d_sid, spec_sid_EE[n_sweep2], ':g', linewidth=.5, label='EE(SID-3) mid')
ax4.plot(freq_1d_sid, spec_sid_EE[n_sweep3], ':k', linewidth=.5, label='EE(SID-3) last')
ax4.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep1], '-r', linewidth=.9, label='EE_msk(SID-3) 1st')
ax4.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep2], '-g', linewidth=.5, label='EE_msk(SID-3) mid')
ax4.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep3], '-k', linewidth=.5, label='EE_msk(SID-3) last')
ax1.set_yscale('log'); ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log')

# Label
ax1.set_ylabel('EE '+power_str);       ax2.set_ylabel('EE-msk '+power_str)
ax3.set_ylabel('EE:SID-2 '+power_str); ax4.set_ylabel('EE:SID-3 emu '+power_str); ax4.set_xlabel('Frequency [kHz]')
#
date1 = Epoch_1d[n_sweep1];  date1 = date1.strftime('1st: %Y/%m/%d %R:%S   ')
date2 = Epoch_1d[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = Epoch_1d[n_sweep3];  date3 = date3.strftime('Fin: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min, f_max]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim)

# range: Y-axis
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-sid3emu.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-sid3emu-log.png'
    fig.savefig(png_fname)

In [ ]:
# Sweep_num
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
n_sweep1 = n_time0//4;  n_sweep2 = 3*n_time0//4;  n_sweep3 = n_time0-1
# n_sweep1 = 50; n_sweep2 = 1600;  n_sweep3 = 2000
# Y-range
p_min = p_min0;  p_max = p_max0+1
# X-range
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])
#   f_min = 3900;        f_max = 5100;        print(data.frequency[0][0][0], data.frequency[0][8][0])

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.9, label='EE(SID-2) 1st')
ax1.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.5, label='EE(SID-2) mid')
ax1.plot(freq_1d, spec.EE[n_sweep3], ':k', linewidth=.5, label='EE(SID-2) last')
ax1.plot(freq_1d_sid, spec_sid_EE_med[n_sweep1], '-r', linewidth=.9, label='EE(SID-3) 1st')
ax1.plot(freq_1d_sid, spec_sid_EE_med[n_sweep2], '-g', linewidth=.5, label='EE(SID-3) mid')
ax1.plot(freq_1d_sid, spec_sid_EE_med[n_sweep3], '-k', linewidth=.5, label='EE(SID-3) last')
#
ax2.plot(freq_1d_msk, spec_msk.EE[n_sweep1], ':r', linewidth=.9, label='EE_msk(SID-2) 1st')
ax2.plot(freq_1d_msk, spec_msk.EE[n_sweep2], ':g', linewidth=.5, label='EE_msk(SID-2) mid')
ax2.plot(freq_1d_msk, spec_msk.EE[n_sweep3], ':k', linewidth=.5, label='EE_msk(SID-2) last')
ax2.plot(freq_1d_sid, spec_sid_EE_msk_med[n_sweep1], '-r', linewidth=.9, label='EE_msk(SID-3) 1st')
ax2.plot(freq_1d_sid, spec_sid_EE_msk_med[n_sweep2], '-g', linewidth=.5, label='EE_msk(SID-3) mid')
ax2.plot(freq_1d_sid, spec_sid_EE_msk_med[n_sweep3], '-k', linewidth=.5, label='EE_msk(SID-3) last')
#
ax3.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.9, label='EE(SID-2) 1st')
ax3.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.5, label='EE(SID-2) mid')
ax3.plot(freq_1d, spec.EE[n_sweep3], ':k', linewidth=.3, label='EE(SID-2) last')
ax3.plot(freq_1d_msk, spec_msk.EE[n_sweep1], '-r', linewidth=.9, label='EE_msk(SID-2) 1st')
ax3.plot(freq_1d_msk, spec_msk.EE[n_sweep2], '-g', linewidth=.5, label='EE_msk(SID-2) mid')
ax3.plot(freq_1d_msk, spec_msk.EE[n_sweep3], '-k', linewidth=.5, label='EE_msk(SID-2) last')
#
#ax4.plot(freq_1d_sid, spec_sid_EE_med[n_sweep1], ':r', linewidth=.9, label='EE(SID-3) 1st')
#ax4.plot(freq_1d_sid, spec_sid_EE_med[n_sweep2], ':g', linewidth=.5, label='EE(SID-3) mid')
#ax4.plot(freq_1d_sid, spec_sid_EE_med[n_sweep3], ':k', linewidth=.5, label='EE(SID-3) last')
ax4.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep1], ':r', linewidth=.9, label='EE(SID-3) 1st')
ax4.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep2], ':g', linewidth=.5, label='EE(SID-3) mid')
ax4.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep3], ':k', linewidth=.5, label='EE(SID-3) last')
ax4.plot(freq_1d_sid, spec_sid_EE_msk_med[n_sweep1], '-r', linewidth=.9, label='EE_msk(SID-3) 1st')
ax4.plot(freq_1d_sid, spec_sid_EE_msk_med[n_sweep2], '-g', linewidth=.5, label='EE_msk(SID-3) mid')
ax4.plot(freq_1d_sid, spec_sid_EE_msk_med[n_sweep3], '-k', linewidth=.5, label='EE_msk(SID-3) last')
ax1.set_yscale('log'); ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log')

# Label
ax1.set_ylabel('EE '+power_str);       ax2.set_ylabel('EE-msk '+power_str)
ax3.set_ylabel('EE:SID-2 '+power_str); ax4.set_ylabel('EE:SID-3 emu '+power_str); ax4.set_xlabel('Frequency [kHz]')
#
date1 = Epoch_1d[n_sweep1];  date1 = date1.strftime('1st: %Y/%m/%d %R:%S   ')
date2 = Epoch_1d[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = Epoch_1d[n_sweep3];  date3 = date3.strftime('Fin: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min, f_max]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim)

# range: Y-axis
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-sid3emu_med.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-sid3emu-log_med.png'
    fig.savefig(png_fname)

### FT - SID-3 emulation

In [ ]:
# Y-range
f_min = f_min0;      f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])
#   f_min = 4900;        f_max = 6100;        print(data.frequency[0][0][0], data.frequency[0][8][0])
# Z-range
p_min  = p_min0+1.5;     p_max  = p_max0-2
p_min2 = p_min0+1.5;     p_max2 = p_max0-2

fig2d = plt.figure(figsize=[12,16])
ax1 = fig2d.add_subplot(5, 1, 1);  ax2 = fig2d.add_subplot(5, 1, 2); ax3 = fig2d.add_subplot(5, 1, 3); ax4 = fig2d.add_subplot(5, 1, 4)
if time_mode == 1:
    ax5 = fig2d.add_subplot(5, 1, 5)

# Y-axis
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log');        ax4.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]'); ax4.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title('EE');  ax2.set_title('EE-msk');  ax3.set_title('EE:SID-3 emu'); ax4.set_title('EE-msk:SID-3 emu')
if time_mode == 1:
    ax5.set_xlabel(str_date)
else:
    ax4.set_xlabel(str_date)

# Plot
print(num_1d.shape, freq_1d.shape, EE_2d.shape)
if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d,     EE_2d,         norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d_msk, EE_msk_2d,     norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d_sid, EE_sid_2d,     norm=colors.LogNorm(vmin=10**p_min2, vmax=10**p_max2), cmap='jet')
    p4 = ax4.pcolormesh(num_1d, freq_1d_sid, EE_sid_2d_msk, norm=colors.LogNorm(vmin=10**p_min2, vmax=10**p_max2), cmap='jet')
    p5 = ax5.plot(np.ravel(Epoch_1d[:]), '.')
    pp4 = fig2d.colorbar(p4, ax=ax5, orientation="vertical"); pp4.set_label(power_str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d,     EE_2d,         norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d_msk, EE_msk_2d,     norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d_sid, EE_sid_2d,     norm=colors.LogNorm(vmin=10**p_min2, vmax=10**p_max2), cmap='jet')
    p4 = ax4.pcolormesh(Epoch_1d, freq_1d_sid, EE_sid_2d_msk, norm=colors.LogNorm(vmin=10**p_min2, vmax=10**p_max2), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(power_str)
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical"); pp4.set_label(power_str)

# range: X-axis
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)

# range: Y-axis
"""
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)
"""

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT-sid3emu.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-sid3emu-log.png'
    fig2d.savefig(png_fname)

In [ ]:
# Y-range
f_min = f_min0;      f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])
#   f_min = 4900;        f_max = 6100;        print(data.frequency[0][0][0], data.frequency[0][8][0])
# Z-range
p_min  = p_min0+1.5;     p_max  = p_max0-2

fig2d = plt.figure(figsize=[12,16])
ax1 = fig2d.add_subplot(5, 1, 1);  ax2 = fig2d.add_subplot(5, 1, 2); ax3 = fig2d.add_subplot(5, 1, 3); ax4 = fig2d.add_subplot(5, 1, 4)
if time_mode == 1:
    ax5 = fig2d.add_subplot(5, 1, 5)

# Y-axis
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log');        ax4.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]'); ax4.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title('EE_mean');  ax2.set_title('EE-msk_mean');  ax3.set_title('EE_med'); ax4.set_title('EE-msk_med')
if time_mode == 1:
    ax5.set_xlabel(str_date)
else:
    ax4.set_xlabel(str_date)

# Plot
print(num_1d.shape, freq_1d.shape, EE_2d.shape)
if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d_sid, EE_sid_2d_mean,     norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d_sid, EE_sid_2d_msk_mean, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d_sid, EE_sid_2d_med,      norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.pcolormesh(num_1d, freq_1d_sid, EE_sid_2d_msk_med,  norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p5 = ax5.plot(np.ravel(Epoch_1d[:]), '.')
    pp4 = fig2d.colorbar(p4, ax=ax5, orientation="vertical"); pp4.set_label(power_str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d_sid, EE_sid_2d_mean,     norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d_sid, EE_sid_2d_msk_mean, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d_sid, EE_sid_2d_med,      norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.pcolormesh(Epoch_1d, freq_1d_sid, EE_sid_2d_msk_med,  norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(power_str)
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical"); pp4.set_label(power_str)

# range: X-axis
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)

# range: Y-axis
"""
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)
"""

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT-sid3emu_med.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-sid3emu-log_med.png'
    fig2d.savefig(png_fname)

# SID-4 & SID-20 emulation

In [ ]:
"""
# SID-4 & SID-20: Frequency
freq_1d_sid, f_step_sid, f_width_sid = juice_cdf._frequency_sid4_sid20()
n_freq1_sid = len(freq_1d_sid)
n_freq1 = len(freq_1d)
n_freq1_msk = len(freq_1d_msk)
tbl_freq_to_sid     = juice_cdf._frequency_sid2_to_data(freq_1d_sid, f_step_sid, freq_1d)
tbl_freq_to_sid_msk = juice_cdf._frequency_sid2_to_data(freq_1d_sid, f_step_sid, freq_1d_msk)

spec_sid_EE = np.zeros((n_time0, n_freq1_sid))
spec_sid_EE_msk = np.zeros((n_time0, n_freq1_sid))
#print(spec.EE.shape, n_time0, n_freq1)
#print(spec_msk.EE.shape, n_time0, n_freq1_msk, spec.EE.shape, spec_msk.EE.shape, n_freq1, n_freq1_msk)

for i in range(n_time0):
    for j in range(n_freq1_sid):
        j0 = tbl_freq_to_sid[j][0]; j0_msk = tbl_freq_to_sid_msk[j][0]
        j1 = tbl_freq_to_sid[j][1]; j1_msk = tbl_freq_to_sid_msk[j][1]
        j2 = tbl_freq_to_sid[j][2]; j2_msk = tbl_freq_to_sid_msk[j][2]
        spec_sid_EE[i][j]     = sum(spec.EE[i][j0:j1+1])
        spec_sid_EE_msk[i][j] = sum(spec_msk.EE[i][j0_msk:j1_msk+1])
        #if i==0:
        #    print( j, j0, j1, j2, spec_sid_EE[i][j], spec_sid_EE_msk[i][j], j0_msk, j1_msk, j2_msk, spec.EE[i][j0], spec_msk.EE[i][j0_msk])

EE_sid_2d = spec_sid_EE.transpose();   EE_sid_2d_msk = spec_sid_EE_msk.transpose()
"""
"""
mask_file = './test.csv'
with open(mask_file, 'w') as f:
    writer = csv.writer(f)
    for i in range(n_freq1_sid):
        f_min = freq_1d_sid[i] - f_step_sid[i]/2
        f_max = freq_1d_sid[i] + f_step_sid[i]/2
        writer.writerow([ i, f_min, f_max, tbl_freq_to_sid[i, 0], tbl_freq_to_sid[i, 1], tbl_freq_to_sid[i, 2], tbl_freq_to_sid_msk[i, 0], tbl_freq_to_sid_msk[i, 1], tbl_freq_to_sid_msk[i, 2] ])
print("sample write: ", mask_list.shape, mask_file)
"""

### Spectrum - SID-4 & SID-20 emulation

In [ ]:
"""
# Sweep_num
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
# Y-range
p_min = p_min0;  p_max = p_max0
# X-range
f_min = freq_1d_sid[0];  f_max = freq_1d_sid[-1]

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.9, label='EE(SID-2) 1st')
ax1.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.5, label='EE(SID-2) mid')
ax1.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.3, label='EE(SID-2) last')
ax1.plot(freq_1d_sid, spec_sid_EE[n_sweep1], '-r', linewidth=.9, label='EE(SID-4) 1st')
ax1.plot(freq_1d_sid, spec_sid_EE[n_sweep2], '-g', linewidth=.5, label='EE(SID-4) mid')
ax1.plot(freq_1d_sid, spec_sid_EE[n_sweep3], '-b', linewidth=.3, label='EE(SID-4) last')
#
ax2.plot(freq_1d_msk, spec_msk.EE[n_sweep1], ':r', linewidth=.9, label='EE_msk(SID-2) 1st')
ax2.plot(freq_1d_msk, spec_msk.EE[n_sweep2], ':g', linewidth=.5, label='EE_msk(SID-2) mid')
ax2.plot(freq_1d_msk, spec_msk.EE[n_sweep3], ':b', linewidth=.3, label='EE_msk(SID-2) last')
ax2.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep1], '-r', linewidth=.9, label='EE_msk(SID-4) 1st')
ax2.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep2], '-g', linewidth=.5, label='EE_msk(SID-4) mid')
ax2.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep3], '-b', linewidth=.3, label='EE_msk(SID-4) last')
#
ax3.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.9, label='EE(SID-2) 1st')
ax3.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.5, label='EE(SID-2) mid')
ax3.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.3, label='EE(SID-2) last')
ax3.plot(freq_1d_msk, spec_msk.EE[n_sweep1], '-r', linewidth=.9, label='EE_msk(SID-2) 1st')
ax3.plot(freq_1d_msk, spec_msk.EE[n_sweep2], '-g', linewidth=.5, label='EE_msk(SID-2) mid')
ax3.plot(freq_1d_msk, spec_msk.EE[n_sweep3], '-b', linewidth=.3, label='EE_msk(SID-2) last')
#
ax4.plot(freq_1d_sid, spec_sid_EE[n_sweep1], ':r', linewidth=.9, label='EE(SID-4) 1st')
ax4.plot(freq_1d_sid, spec_sid_EE[n_sweep2], ':g', linewidth=.5, label='EE(SID-4) mid')
ax4.plot(freq_1d_sid, spec_sid_EE[n_sweep3], ':b', linewidth=.3, label='EE(SID-4) last')
ax4.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep1], '-r', linewidth=.9, label='EE_msk(SID-4) 1st')
ax4.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep2], '-g', linewidth=.5, label='EE_msk(SID-4) mid')
ax4.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep3], '-b', linewidth=.3, label='EE_msk(SID-4) last')
ax1.set_yscale('log'); ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log')
# ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log')

# Label
ax1.set_ylabel('EE '+power_str);       ax2.set_ylabel('EE-msk '+power_str)
ax3.set_ylabel('EE:SID-2 '+power_str); ax4.set_ylabel('EE:SID-4 emu '+power_str); ax4.set_xlabel('Frequency [kHz]')
#
date1 = Epoch_1d[n_sweep1];  date1 = date1.strftime('1st: %Y/%m/%d %R:%S   ')
date2 = Epoch_1d[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = Epoch_1d[n_sweep3];  date3 = date3.strftime('Fin: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min, f_max]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim)

# range: Y-axis
ylim=[10**(p_min+1), 10**(p_max)]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-sid4emu.png'
    fig.savefig(png_fname)
"""

### FT - SID-4 & SID-20 emulation

In [ ]:
"""
# Y-range
f_min = freq_1d_sid[0];  f_max = freq_1d_sid[-1]
# Z-range
p_min  = p_min0+1.5;     p_max  = p_max0-2.5
p_min2 = p_min0+2;       p_max2 = p_max0-2

fig2d = plt.figure(figsize=[12,16])
ax1 = fig2d.add_subplot(5, 1, 1);  ax2 = fig2d.add_subplot(5, 1, 2); ax3 = fig2d.add_subplot(5, 1, 3); ax4 = fig2d.add_subplot(5, 1, 4)
if time_mode == 1:
    ax5 = fig2d.add_subplot(5, 1, 5)

# Y-axis
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max)
# ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log');        ax4.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]'); ax4.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title('EE');  ax2.set_title('EE-msk');  ax3.set_title('EE:SID-4 emu'); ax4.set_title('EE-msk:SID-4 emu')
if time_mode == 1:
    ax5.set_xlabel(str_date)
else:
    ax4.set_xlabel(str_date)

# Plot
print(num_1d.shape, freq_1d.shape, EE_2d.shape)
if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d,     EE_2d,         norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d_msk, EE_msk_2d,     norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d_sid, EE_sid_2d,     norm=colors.LogNorm(vmin=10**p_min2, vmax=10**p_max2), cmap='jet')
    p4 = ax4.pcolormesh(num_1d, freq_1d_sid, EE_sid_2d_msk, norm=colors.LogNorm(vmin=10**p_min2, vmax=10**p_max2), cmap='jet')
    p5 = ax5.plot(np.ravel(Epoch_1d[:]), '.')
    pp4 = fig2d.colorbar(p4, ax=ax5, orientation="vertical"); pp4.set_label(power_str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d,     EE_2d,         norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d_msk, EE_msk_2d,     norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d_sid, EE_sid_2d,     norm=colors.LogNorm(vmin=10**p_min2, vmax=10**p_max2), cmap='jet')
    p4 = ax4.pcolormesh(Epoch_1d, freq_1d_sid, EE_sid_2d_msk, norm=colors.LogNorm(vmin=10**p_min2, vmax=10**p_max2), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(power_str)
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical"); pp4.set_label(power_str)

# range: X-axis
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)

# range: Y-axis
#ylim=[10**(p_min), 10**(p_max)]
#ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT-sid4emu.png'
    fig2d.savefig(png_fname)
"""


# SID-5 & SID-21 emulation

In [ ]:
"""
# SID-5 & SID-21: Frequency
freq_1d_sid, f_step_sid, f_width_sid = juice_cdf._frequency_sid5_sid21()
n_freq1_sid = len(freq_1d_sid)
n_freq1 = len(freq_1d)
n_freq1_msk = len(freq_1d_msk)
tbl_freq_to_sid     = juice_cdf._frequency_sid2_to_data(freq_1d_sid, f_step_sid, freq_1d)
tbl_freq_to_sid_msk = juice_cdf._frequency_sid2_to_data(freq_1d_sid, f_step_sid, freq_1d_msk)

spec_sid_EE = np.zeros((n_time0, n_freq1_sid))
spec_sid_EE_msk = np.zeros((n_time0, n_freq1_sid))

for i in range(n_time0):
    for j in range(n_freq1_sid):
        j0 = tbl_freq_to_sid[j][0]; j0_msk = tbl_freq_to_sid_msk[j][0]
        j1 = tbl_freq_to_sid[j][1]; j1_msk = tbl_freq_to_sid_msk[j][1]
        j2 = tbl_freq_to_sid[j][2]; j2_msk = tbl_freq_to_sid_msk[j][2]
        spec_sid_EE[i][j]     = sum(spec.EE[i][j0:j1+1])
        spec_sid_EE_msk[i][j] = sum(spec_msk.EE[i][j0_msk:j1_msk+1])
        #if i==0:
        #    print( j, j0, j1, j2, spec_sid_EE[i][j], spec_sid_EE_msk[i][j], j0_msk, j1_msk, j2_msk, spec.EE[i][j0], spec_msk.EE[i][j0_msk])

EE_sid_2d = spec_sid_EE.transpose();   EE_sid_2d_msk = spec_sid_EE_msk.transpose()
"""
"""
mask_file = './test.csv'
with open(mask_file, 'w') as f:
    writer = csv.writer(f)
    for i in range(n_freq1_sid):
        f_min = freq_1d_sid[i] - f_step_sid[i]/2
        f_max = freq_1d_sid[i] + f_step_sid[i]/2
        writer.writerow([ i, f_min, f_max, tbl_freq_to_sid[i, 0], tbl_freq_to_sid[i, 1], tbl_freq_to_sid[i, 2], tbl_freq_to_sid_msk[i, 0], tbl_freq_to_sid_msk[i, 1], tbl_freq_to_sid_msk[i, 2] ])
print("sample write: ", mask_list.shape, mask_file)
"""

### Spectrum - SID-5 & SID-21 emulation

In [ ]:
"""
# Sweep_num
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
# Y-range
p_min = p_min0;  p_max = p_max0
# X-range
f_min = freq_1d_sid[0];  f_max = freq_1d_sid[-1]
if f_mode == 0:
    f_min = f_min0;  f_max = 5000

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.2, label='EE(SID-2) 1st')
ax1.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.2, label='EE(SID-2) mid')
ax1.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.2, label='EE(SID-2) last')
ax1.plot(freq_1d_sid, spec_sid_EE[n_sweep1], '-r', linewidth=1, label='EE(SID-5) 1st')
ax1.plot(freq_1d_sid, spec_sid_EE[n_sweep2], '-g', linewidth=1, label='EE(SID-5) mid')
ax1.plot(freq_1d_sid, spec_sid_EE[n_sweep3], '-b', linewidth=1, label='EE(SID-5) last')
#
ax2.plot(freq_1d_msk, spec_msk.EE[n_sweep1], ':r', linewidth=.2, label='EE_msk(SID-2) 1st')
ax2.plot(freq_1d_msk, spec_msk.EE[n_sweep2], ':g', linewidth=.2, label='EE_msk(SID-2) mid')
ax2.plot(freq_1d_msk, spec_msk.EE[n_sweep3], ':b', linewidth=.2, label='EE_msk(SID-2) last')
ax2.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep1], '-r', linewidth=1, label='EE_msk(SID-5) 1st')
ax2.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep2], '-g', linewidth=1, label='EE_msk(SID-5) mid')
ax2.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep3], '-b', linewidth=1, label='EE_msk(SID-5) last')
#
ax3.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.2, label='EE(SID-2) 1st')
ax3.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.2, label='EE(SID-2) mid')
ax3.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.2, label='EE(SID-2) last')
ax3.plot(freq_1d_msk, spec_msk.EE[n_sweep1], '-r', linewidth=1, label='EE_msk(SID-2) 1st')
ax3.plot(freq_1d_msk, spec_msk.EE[n_sweep2], '-g', linewidth=1, label='EE_msk(SID-2) mid')
ax3.plot(freq_1d_msk, spec_msk.EE[n_sweep3], '-b', linewidth=1, label='EE_msk(SID-2) last')
#
ax4.plot(freq_1d_sid, spec_sid_EE[n_sweep1], ':r', linewidth=.2, label='EE(SID-5) 1st')
ax4.plot(freq_1d_sid, spec_sid_EE[n_sweep2], ':g', linewidth=.2, label='EE(SID-5) mid')
ax4.plot(freq_1d_sid, spec_sid_EE[n_sweep3], ':b', linewidth=.2, label='EE(SID-5) last')
ax4.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep1], '-r', linewidth=1, label='EE_msk(SID-5) 1st')
ax4.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep2], '-g', linewidth=1, label='EE_msk(SID-5) mid')
ax4.plot(freq_1d_sid, spec_sid_EE_msk[n_sweep3], '-b', linewidth=1, label='EE_msk(SID-5) last')
ax1.set_yscale('log'); ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log')

# Label
ax1.set_ylabel('EE '+power_str);       ax2.set_ylabel('EE-msk '+power_str)
ax3.set_ylabel('EE:SID-2 '+power_str); ax4.set_ylabel('EE:SID-5 emu '+power_str); ax4.set_xlabel('Frequency [kHz]')
#
date1 = Epoch_1d[n_sweep1];  date1 = date1.strftime('First: %Y/%m/%d %R:%S   ')
date2 = Epoch_1d[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = Epoch_1d[n_sweep3];  date3 = date3.strftime('Last: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min, f_max]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim)

# range: Y-axis
# ylim=[10**(p_min), 10**(p_max)]
# ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-sid5emu.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-sid5emu-log.png'
    fig.savefig(png_fname)
"""

### FT - SID-5 & SID-21 emulation

In [ ]:
"""
# Y-range
f_min = freq_1d_sid[0];  f_max = freq_1d_sid[-1]
# Z-range
p_min  = p_min0+1;    p_max  = p_max0-3
p_min2 = p_min0+2;    p_max2 = p_max0
#f_mode = 0;   f_min = 1200;  f_max = 1600;  print(data.frequency[0][0][0], data.frequency[0][8][0])

fig2d = plt.figure(figsize=[12,16])
ax1 = fig2d.add_subplot(5, 1, 1);  ax2 = fig2d.add_subplot(5, 1, 2); ax3 = fig2d.add_subplot(5, 1, 3); ax4 = fig2d.add_subplot(5, 1, 4)
if time_mode == 1:
    ax5 = fig2d.add_subplot(5, 1, 5)

# Y-axis
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log');        ax4.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]'); ax4.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title('EE');  ax2.set_title('EE-msk');  ax3.set_title('EE:SID-5 emu'); ax4.set_title('EE-msk:SID-5 emu')
if time_mode == 1:
    ax5.set_xlabel(str_date)
else:
    ax4.set_xlabel(str_date)

# Plot
print(num_1d.shape, freq_1d.shape, EE_2d.shape)
if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d,     EE_2d,         norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d_msk, EE_msk_2d,     norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d_sid, EE_sid_2d,     norm=colors.LogNorm(vmin=10**p_min2, vmax=10**p_max2), cmap='jet')
    p4 = ax4.pcolormesh(num_1d, freq_1d_sid, EE_sid_2d_msk, norm=colors.LogNorm(vmin=10**p_min2, vmax=10**p_max2), cmap='jet')
    p5 = ax5.plot(np.ravel(Epoch_1d[:]), '.')
    pp4 = fig2d.colorbar(p4, ax=ax5, orientation="vertical"); pp4.set_label(power_str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d,     EE_2d,         norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d_msk, EE_msk_2d,     norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d_sid, EE_sid_2d,     norm=colors.LogNorm(vmin=10**p_min2, vmax=10**p_max2), cmap='jet')
    p4 = ax4.pcolormesh(Epoch_1d, freq_1d_sid, EE_sid_2d_msk, norm=colors.LogNorm(vmin=10**p_min2, vmax=10**p_max2), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(power_str)
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical"); pp4.set_label(power_str)

# range: X-axis
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)

# range: Y-axis
# ylim=[10**(p_min), 10**(p_max)]
# ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT-sid5emu.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-sid5emu-log.png'
    fig2d.savefig(png_fname)
"""